In [ ]:
# !pip3 install tqdm requests beautifulsoup4 selenium webdriver_manager

In [ ]:
import os
import re
import string
import time
from tqdm.notebook import tqdm
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from webdriver_manager.microsoft import EdgeChromiumDriverManager

# Set URL

In [ ]:
playlist = 1111
URL = f"https://www.aparat.com/playlist/{playlist}"

# Get Page

In [ ]:
def get_soup(URL, driver='Chrome'):
    if driver=='Chrome':   
        driver = webdriver.Chrome(ChromeDriverManager().install())
    elif driver=='Firefox':
        driver = webdriver.Firefox(GeckoDriverManager().install())
    elif driver=='Edge':
        driver = webdriver.Edge(EdgeChromiumDriverManager().install())
    
    driver.get(URL)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")    
    return soup
#
soup = get_soup(URL)

# Create Folder

In [ ]:
title = soup.find('div', attrs={'class': 'playlist-field'}).get_text()
path = 'E:\\Video_DL\\' + title

try:
    os.makedirs(path)
    print(path)
    print('This folder is created')
except:
    print(path)
    print('This folder is exist')

#    
os.chdir(path)  

# Downlaod Courses

In [ ]:
download_urls = []
playlist_items = soup.find_all('li', attrs={'class': 'playlist-item'})

x = "۰۱۲۳۴۵۶۷۸۹"
y = "0123456789"
table_digit = str.maketrans(x, y)
table_punctuation = str.maketrans(string.punctuation, '_'*len(string.punctuation))

with tqdm(total  =len(playlist_items)) as pbar:
    for index, a in enumerate(playlist_items):
        isok = True
        while isok:
            try:
                a_tag = a.find('div', attrs={'class': 'thumb-content'}).find('a', href=True, text=True)
                item_url = 'https://www.aparat.com' + a_tag['href']
                item_title = a_tag.get_text()
                print(item_title)
                
                file_name = str(index+1) + '_' + item_title.translate(table_punctuation)              
                file_name = file_name.translate(table_digit) + '.mp4'                              
                   
                if os.path.exists(file_name)==False:
                    item_soup = get_soup(item_url)
                    file_url = item_soup.find("meta", property="og:video")['content']

                    print(file_name)
                    print(file_url)

                    download_urls.append(file_url)
                    r = requests.get(file_url, stream = True) 
                    
                    os.chdir(path)
                    with open(file_name,"wb") as video: 
                        for chunk in r.iter_content(chunk_size=1024):         
                            if chunk: 
                                video.write(chunk) 
                isok = False
            except :
                print('Error')
                isok = True
        pbar.update(1)